## 파이썬 Folium을 이용해서 지도 시각화를 하는 방법
---
파이썬(Python)에서는 위경도 데이터를 이용해서 지도 위에 시각화를 할 수 있습니다. 이런 작업을 할 수 있는 패키지로는 대표적으로 폴리움(Folium)이 존재하는데, 이를 이용해서 오픈스트리트맵(OpenStreetMap) 기반의 지도 위에 GeoJSON 파일에 기록된 위경도 데이터를 기준으로 표기를 해 보는 방법을 알아보겠습니다.  
</p></br></br>


이번 시각화에는 서울특별시 행정동 지도를 GeoJSON 파일로 정리한 것, 그중에서 송파구에 해당하는 정보를 이용해 위경도 정보를 얻어보겠습니다. 표기할 정보는 송파구의 인구 대비 코로나19 확진자 정보입니다.  
</p></br></br>


* 행정동 정보: https://github.com/vuski/admdongkor/blob/master/ver20230401/HangJeongDong_ver20230401.geojson  
</p></br></br>



In [1]:
# 패키지 불러오기
import folium
import json
import pandas as pd

df = pd.read_csv('./data/인구대비_확진자.csv')  # 확진자 정보 불러오기
songpa_geo = json.load(open('./data/songpa.geojson.txt', encoding='utf8'))  # GeoJSON 불러오기

</p></br></br>


인구 대비 확진자 정보의 경우, 행정동명과 2022년 확진자 비율이 적혀있는 판다스 데이터프레임(Pandas DataFrame) 형식으로 준비되어 있습니다. 이 파일에는 위경도 정보가 명시되어 있지 않은데, 행정동 정보 GeoJSON 파일에 행정동명 정보가 있기 때문에 이를 기준으로 위경도 정보를 찾아보겠습니다.  
</p></br></br>



In [2]:
songpa_df = pd.DataFrame(
    [[i['properties']['adm_nm'][10:], i['properties']['adm_cd']] for i in songpa_geo['features']]
)

songpa_df.columns = ['행정동명', '고유번호']
df_merge = pd.merge(df, songpa_df, left_on='행정동명', right_on='행정동명')

df_merge.head()

,행정동명,2022,고유번호
0,풍납1동,0.5506,1124051
1,풍납2동,0.5468,1124052
2,거여1동,0.5482,1124053
3,마천2동,0.5015,1124056
4,오륜동,0.5579,1124059


</p></br></br>


GeoJSON 데이터의 구조는 파이썬 딕셔너리(Dictionary) 자료형과 같이 처리할 수 있습니다. 우리는 행정동 정보를 추출해야 하는데, 이를 위해서는 해당 파일 내부에 `features` 정보를 참고해야 합니다. 각각의 `features` 정보 안에는 `properties` 정보가 있고, 그 안에 `adm_nm` 정보가 있는데, 이게 바로 주소입니다. 우리는 주소 정보 중에서 행정동에 해당하는 내용만 잘라서 사용하겠습니다.  
</p></br></br>


그리고, GeoJSON 정보를 참조할 때 기준으로 활용할 고유번호를 추가로 저장합니다. 이는 각각의 `features` 정보 안에 있는 `properties` 내부의 `adm_cd` 정보를 활용하겠습니다.  
</p></br></br>



In [3]:
title = '송파구 2022년 확진자'
title_html = f'<h3 style="font-size:16px"><b>{title}</b></h3>' 

</p></br></br>


다음으로는 지도 제목을 지정해 줍니다. HTML 태그를 이용해서 적절한 크기 및 양식을 맞춰주면 되는데, 위의 코드를 이용한다면 16픽셀 사이즈의 적당한 제목이 작성됩니다.  
</p></br></br>



In [4]:
m = folium.Map(
    location=[37.5,127.1],
    zoom_start=12, 
    width=640,
    height=480,
)

# 지도 제목 추가
m.get_root().html.add_child(folium.Element(title_html))

</p></br></br>


파이썬 폴리움에서 지도의 초기 위치 설정은 `folium.Map` 함수를 이용합니다. 위경도 정보와 확대 기준, 그리고 지도 크기를 설정하면 시각화가 가능한데, 저는 송파구를 가운데에 두기 위해 위경도는 **3.5, 127.1** 로, 확대 기준은 **12** 로, 지도 크기는 **640\*480px** 으로 설정했습니다.  
</p></br></br>



In [5]:
folium.GeoJson(
    songpa_geo,
    name='행정동명'
).add_to(m)

</p></br></br>


다음으로는 행정동의 경계를 따라서 도형을 그려주는 작업입니다. 행정동의 경계 정보는 GeoJSON에 명시되어 있기 때문에, `folium.GeoJson` 함수를 이용하면 간단하게 시각화가 가능합니다. 해당 함수는 GeoJSON 파일과 표기할 내용의 이름을 설정해 주면 됩니다.  
</p></br></br>



In [6]:
#지도에 색 적용 및 데이터 연결
choropleth =  folium.Choropleth(
    geo_data=songpa_geo,
    name=df_merge,
    data=df_merge,
    columns=('고유번호', '2022'),
    key_on='feature.properties.adm_cd',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='확진자 비율',
    nan_fill_color='white',
    nan_fill_opacity=0
).add_to(m)

</p></br></br>


다음으로는 보유하고 있는 데이터를 행정동 단위로 색칠해주는 작업을 진행하겠습니다. 해당 작업은 `folium.Choropleth` 함수를 이용하는데, 이는 입력할 패러미터가 많기 때문에 함수 사용 전 꼼꼼하게 확인해 두시는걸 추천드립니다. 여기에 입력한 패러미터는 아래 내용을 참고해 주세요.  
</p></br></br>


* `geo_data`: GeoJSON 파일
* `name`, `data`: 표기할 데이터프레임
* `columns`: GeoJSON을 참조할 대상, 지도에 색칠할 대상
* `key_on`: GeoJSON에서 참조되는 대상
* `fill_color`: 지도에서 색칠할 컬러 팔레트
* `fill_opacity`: 색칠되는 부분의 알파값(투명도)
* `line_opacity`: 테두리 부분의 알파값(투명도)
* `legend_name`: 지도에 표기할 범례
* `nan_fill_color`: NaN(결측치) 값의 경우 색칠할 색깔
* `nan_fill_opacity`: NaN(결측치) 값의 경우 알파값(투명도)  
</p></br></br>


이렇게 지도를 작성하면 아래와 같이 확인할 수 있습니다.  
</p></br></br>



In [7]:
m

</p></br></br>


만약, 지도 시각화 데이터를 HTML 파일로 내보내고 싶다면, `save` 함수를 이용해서 내보내기할 수 있습니다.  
</p></br></br>



In [8]:
m.save('./res.html')